In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pefile
!pip install lief==0.12.2
!pip install jupyter vega vega_datasets altair matplotlib

In [ ]:
import os
import pandas as pd
import numpy as np
import lief

# Lưu đường dẫn tới các file PE thành arm.txt

In [ ]:
thu_muc_mau = '/content/drive/MyDrive/UNZIP_BODMAS/altered'

ten_tep = '/content/drive/MyDrive/UNZIP_BODMAS/arm.txt'

if not os.path.exists(thu_muc_mau):
    print(f"Thư mục '{thu_muc_mau}' không tồn tại.")
else:
    # Lấy danh sách các đường dẫn tệp trong thư mục
    danh_sach_duong_dan = [os.path.join(thu_muc_mau, tep) for tep in os.listdir(thu_muc_mau) if os.path.isfile(os.path.join(thu_muc_mau, tep))]

    # Mở tệp để ghi
    with open(ten_tep, 'w') as tep:
        # Ghi từng đường dẫn vào tệp, mỗi đường dẫn trên một dòng
        for duong_dan in danh_sach_duong_dan:
            tep.write(f"{duong_dan}\n")

    print(f"Tạo thành công tệp {ten_tep} chứa đường dẫn của mỗi mẫu trong thư mục {thu_muc_mau}.")

Tạo thành công tệp /content/drive/MyDrive/UNZIP_BODMAS/arm.txt chứa đường dẫn của mỗi mẫu trong thư mục /content/drive/MyDrive/UNZIP_BODMAS/altered.


In [ ]:
# kiểm tra số dòng
file_path = "/content/drive/MyDrive/UNZIP_BODMAS/arm.txt"
with open(file_path, 'r') as file:
    line_count = sum(1 for line in file)
print("Number of lines in the file:", line_count)


Number of lines in the file: 18953


# Executable malware binaries

In [ ]:
#Create a folder under the same directory called `armed`, used to save restored binaries.
!nohup python /content/drive/MyDrive/UNZIP_BODMAS/PE_modifier.py --filelist /content/drive/MyDrive/UNZIP_BODMAS/arm.txt --save_record "/content/drive/MyDrive/UNZIP_BODMAS/meta_disarm.csv" --altered_folder /content/drive/MyDrive/UNZIP_BODMAS/armed --arm > /content/drive/MyDrive/UNZIP_BODMAS/restore.log &


nohup: redirecting stderr to stdout


# Extracting feature vectors

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/UNZIP_BODMAS/')
from ember.features import PEFeatureExtractor, ExportsInfo, ExportsInfo, SectionInfo, HeaderFileInfo, GeneralFileInfo, ByteHistogram, ByteEntropyHistogram, ImportsInfo, StringExtractor

In [ ]:
def read_hash_from_filename(file_path):
    return os.path.basename(file_path).replace('.exe', '')

raw_features_dict = {}
def extract_features_from_folder(input_folder, output_file):
    feature_extractor = PEFeatureExtractor()
    metadata_df = pd.read_csv("/content/drive/MyDrive/UNZIP_BODMAS/bodmas_malware_category.csv")
    X = []
    y = []
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            file_path = os.path.join(root, file)

            # Read the content of the PE file
            with open(file_path, 'rb') as f:
                bytez = f.read()

            sha = read_hash_from_filename(file_path)
            # Extract features and label
            features = feature_extractor.feature_vector(bytez)
            if sha in metadata_df['sha256'].values:
                # Retrieve the 'category' value for the corresponding SHA-256 hash
                label = metadata_df.loc[metadata_df['sha256'] == sha, 'category'].values[0]
            else:
                label = '0'
            X.append(features)
            y.append(label)
    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)
    # Save features and labels to npz file
    np.savez(output_file, X=X, y=y)

# Example usage
extract_features_from_folder("/content/drive/MyDrive/UNZIP_BODMAS/armed", "/content/drive/MyDrive/UNZIP_BODMAS/output_features.npz")


# Reading ralated INFO

In [ ]:
file_path='/content/drive/MyDrive/UNZIP_BODMAS/altered/414fc38377f245ab2f92796250835462ae5e8edc95924b541c1a063717dc4dc2.exe'
with open(file_path, 'rb') as f:
  bytez = f.read()
  lief_binary = lief.PE.parse(list(bytez))
  raw_header_info = HeaderFileInfo().raw_features(bytez, lief_binary)
  raw_GeneralFileInfo = GeneralFileInfo().raw_features(bytez, lief_binary)
  raw_SectionInfo = SectionInfo().raw_features(bytez, lief_binary)
  raw_ByteHistogram = ByteHistogram().raw_features(bytez, lief_binary)
  raw_ByteEntropyHistogram = ByteEntropyHistogram().raw_features(bytez, lief_binary)
  raw_ImportsInfo = ImportsInfo().raw_features(bytez, lief_binary)
  raw_StringExtractor = StringExtractor().raw_features(bytez, lief_binary)
raw_header_info

{'coff': {'timestamp': 1524008923,
  'machine': 'UNKNOWN',
  'characteristics': ['CHARA_32BIT_MACHINE', 'EXECUTABLE_IMAGE']},
 'optional': {'subsystem': 'UNKNOWN',
  'dll_characteristics': ['NX_COMPAT', 'TERMINAL_SERVER_AWARE'],
  'magic': 'PE32',
  'major_image_version': 0,
  'minor_image_version': 0,
  'major_linker_version': 10,
  'minor_linker_version': 0,
  'major_operating_system_version': 5,
  'minor_operating_system_version': 1,
  'major_subsystem_version': 5,
  'minor_subsystem_version': 1,
  'sizeof_code': 99840,
  'sizeof_headers': 1024,
  'sizeof_heap_commit': 4096}}

In [ ]:
filename = '/content/drive/MyDrive/UNZIP_BODMAS/output_features.npz'
data = np.load(filename)
X = data['X']
y = data['y']

In [ ]:
unique_labels, counts = np.unique(y, return_counts=True)

for label, count in zip(unique_labels, counts):
    print(f"Label {label}: {count}")

Label backdoor: 469
Label cryptominer: 1
Label downloader: 131
Label dropper: 138
Label informationstealer: 18
Label pua: 2
Label ransomware: 73
Label trojan: 3918
Label virus: 1
Label worm: 1287


In [ ]:
# checking arm and disarm
import os
import pandas as pd

input_folder = '/content/drive/MyDrive/UNZIP_BODMAS/altered'
metadata_path = '/content/drive/MyDrive/UNZIP_BODMAS/Copy of bodmas_metadata.csv'

# Read the metadata CSV into a DataFrame
metadata_df = pd.read_csv(metadata_path)

# Initialize a counter for files with empty 'family' values
empty_family_count = 0

# Iterate through files in the input folder
for root, dirs, files in os.walk(input_folder):
    for file in files:
        file_path = os.path.join(root, file)

        # Extract SHA-256 hash from the file name
        sha = read_hash_from_filename(file_path)

        # Check if the hash exists in the metadata DataFrame
        if sha in metadata_df['sha'].values:
            # Retrieve the 'family' value for the corresponding SHA-256 hash
            family_value = metadata_df.loc[metadata_df['sha'] == sha, 'family'].values[0]

            # Check if 'family' value is empty
            if pd.isna(family_value) or family_value == "":
                empty_family_count += 1

# Print the count of files with empty 'family' values
print("Number of files with empty 'family' values:", empty_family_count)


Number of files with empty 'family' values: 0
